### Packages

In [1]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

2024-04-18 16:23:37.610536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 16:23:37.610667: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 16:23:37.749160: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### llama model

In [27]:
model = AutoModelForCausalLM.from_pretrained("daryl149/llama-2-7b-chat-hf")
tokenizer = AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

### Prompt generation

In [10]:
def prompt_generation(kind, start=None, genre=None, questions=None, context=None):
    if kind == 'question':
        return f'''Context: {start} Genre: '{genre}'. Based on this context and genre generate 3 relevant questions with a $ seperator. Questions: 
        '''
    if kind == 'question2':
        return f'''Context: {start} Using '{genre}' Genre. Generate one small relevant question based on this context and genre. Question: 
        '''
    if kind == 'coherent':
        return f'''You are a story teller. Generate a short 500 words story that utilizes sentences in: "{start}" in {genre} Genre. These sentences need not be sequential and used directly. Dont generate more than 500 words and dont forget the {genre} Genre.
        Story:
        '''

### Zero shot feasibility

In [4]:
sample_starts = ['He likes dancing but he is feeling shy', 'You are a god that hunts other gods', 'Time to go back to school',
                'Last night you were bitten by a werewolf', 'This is going to be my last cigarette.']

In [6]:
start = sample_starts[4]
for i in range(3):
    prompt = prompt_generation(kind='question2', start=start, genre='sad')
    print('Input to model:')
    print(prompt); 
    input_id = torch.tensor(tokenizer(prompt)['input_ids']).reshape(1,-1)
    outs = model.generate(input_ids=input_id, max_length=200)
    text_output = tokenizer.batch_decode(outs.tolist(), skip_special_tokens=True)
    print('Output from model')
    print(text_output[0]); print()
    clean_qs = text_output[0][text_output[0].find('Question:'):].split('Question:')[1].strip(' \n')
    print('Asking user for answer: ')
    answer = input(f'{clean_qs}: '); print()
    start = start + answer
    print('************************************************'); print()

Input to model:
Context: This is going to be my last cigarette. Using 'happy' Genre. Generate one small relevant question based on this context and genre. Question: 
        
Output from model
Context: This is going to be my last cigarette. Using 'happy' Genre. Generate one small relevant question based on this context and genre. Question: 
         As you take your last drag, do you feel a sense of relief wash over you, or are you already missing the familiar feeling of nicotine coursing through your veins?

Asking user for answer: 


As you take your last drag, do you feel a sense of relief wash over you, or are you already missing the familiar feeling of nicotine coursing through your veins?:  I am very sad I dont know how to prevent myself from this habit.



************************************************

Input to model:
Context: This is going to be my last cigarette.I am very sad I dont know how to prevent myself from this habit. Using 'happy' Genre. Generate one small relevant question based on this context and genre. Question: 
        
Output from model
Context: This is going to be my last cigarette.I am very sad I dont know how to prevent myself from this habit. Using 'happy' Genre. Generate one small relevant question based on this context and genre. Question: 
         As you take your last drag, do you think about the countless moments you've missed out on while smoking? 

Asking user for answer: 


As you take your last drag, do you think about the countless moments you've missed out on while smoking?:  My friends used to avoid my company when I smoke.



************************************************

Input to model:
Context: This is going to be my last cigarette.I am very sad I dont know how to prevent myself from this habit.My friends used to avoid my company when I smoke. Using 'happy' Genre. Generate one small relevant question based on this context and genre. Question: 
        
Output from model
Context: This is going to be my last cigarette.I am very sad I dont know how to prevent myself from this habit.My friends used to avoid my company when I smoke. Using 'happy' Genre. Generate one small relevant question based on this context and genre. Question: 
         What if you could replace smoking with a happier habit, like playing a musical instrument or practicing yoga? Would you be more likely to stick to it?

Asking user for answer: 


What if you could replace smoking with a happier habit, like playing a musical instrument or practicing yoga? Would you be more likely to stick to it?:  That is hard but not impossible, will think about it.



************************************************



In [7]:
start

'This is going to be my last cigarette.I am very sad I dont know how to prevent myself from this habit.My friends used to avoid my company when I smoke.That is hard but not impossible, will think about it.'

#### Story generation

In [8]:
def generate_story(start, genre):
    story_gen_prompt1 = prompt_generation(kind='coherent', start=start, genre=genre)
    sample_input2 = torch.tensor(tokenizer(story_gen_prompt1)['input_ids']).reshape(1,-1)
    outs2 = model.generate(input_ids=sample_input2, max_length=300)
    story_output = tokenizer.batch_decode(outs2.tolist(), skip_special_tokens=True)
    print(story_output)
    return story_output

In [9]:
start

'This is going to be my last cigarette.I am very sad I dont know how to prevent myself from this habit.My friends used to avoid my company when I smoke.That is hard but not impossible, will think about it.'

In [11]:
story = generate_story(start, 'sad')

['You are a story teller. Generate a short 500 words story that utilizes sentences in: "This is going to be my last cigarette.I am very sad I dont know how to prevent myself from this habit.My friends used to avoid my company when I smoke.That is hard but not impossible, will think about it." in happy Genre. These sentences need not be sequential and used directly. Dont generate more than 500 words and dont forget the happy Genre.\n        Story:\n         It was a beautiful sunny day and Sarah was sitting on the park bench, enjoying the warmth on her face. She had just lit up her last cigarette and was savoring the moment, knowing it would be her last. She had been trying to quit for months now, but it had been a tough habit to break.\n        As she sat there, she couldn\'t help but feel a sense of sadness wash over her. She had always loved smoking, the way it made her feel relaxed and content. But she knew it was a dangerous habit, one that was slowly killing her. She had lost coun

In [26]:
actual_story = story[0].split('Story:')[1].split('.')
actual_story = [sent.strip('\n ') for sent in actual_story]
actual_story = '. '.join(actual_story)
print('Story:')
actual_story

Story:


"It was a beautiful sunny day and Sarah was sitting on the park bench, enjoying the warmth on her face. She had just lit up her last cigarette and was savoring the moment, knowing it would be her last. She had been trying to quit for months now, but it had been a tough habit to break. As she sat there, she couldn't help but feel a sense of sadness wash over her. She had always loved smoking, the way it made her feel relaxed and content. But she knew it was a dangerous habit, one that was slowly killing her. She had lost count of the number of times she had tried to quit, only to fail miserably. Sarah looked around her and saw a group of children playing nearby. They were laughing and chasing each other, completely carefree. She couldn't help but feel"